<a href="https://colab.research.google.com/github/narsym/Chest-X-Ray-Images-Pneumonia-classification/blob/master/Chest_X_Ray_Images(Pneumonia).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing kaggle to import dataset

In [1]:
!pip install kaggle

Uploading kaggle.json

In [0]:
from google.colab import files
files.upload()

Preparation for the download

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

Downloading the dataset

In [4]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

100% 2.29G/2.29G [00:30<00:00, 31.7MB/s]
100% 2.29G/2.29G [00:31<00:00, 79.4MB/s]


Unzipping the files

In [0]:
!unzip chest-xray-pneumonia.zip

Importing the libraries

In [0]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, optimizers, utils
import cv2
import numpy as np
import os

Directories to load Images

In [0]:
train_dir = './chest_xray/train'
test_dir = './chest_xray/test'
val_dir = './chest_xray/val'

Using image data generator to load and augment the images

In [0]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2
)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

Instantiating the genrators for train, test, validation data

In [10]:
train_generator = train_datagen.flow_from_directory(
        './chest_xray/train',
        target_size=(160, 160),
        batch_size=32,
        class_mode='binary')
val_generator = test_datagen.flow_from_directory(
        './chest_xray/val',
        target_size=(160, 160),
        batch_size=32,
        class_mode='binary')
test_generator = test_datagen.flow_from_directory(
        './chest_xray/test',
        target_size = (160, 160),
        batch_size = 32,
        class_mode = 'binary'
)


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


Finding the number of images in train, test, validation sets

In [0]:
num_train = len(os.listdir('./chest_xray/train/NORMAL')) +  len(os.listdir('./chest_xray/train/PNEUMONIA'))
num_test = len(os.listdir('./chest_xray/test/NORMAL')) + len(os.listdir('./chest_xray/test/PNEUMONIA'))
num_val = len(os.listdir('./chest_xray/val/NORMAL')) + len(os.listdir('./chest_xray/val/PNEUMONIA'))

In [19]:
num_train, num_test, num_val

(5216, 624, 16)

Using pretrained MobileNetV2 model as our base model

In [20]:
IMG_SIZE = 160
IMG_SHAPE = (IMG_SIZE,IMG_SIZE,3)
base_model = tf.keras.applications.MobileNetV2(input_shape = IMG_SHAPE,include_top = False,weights = 'imagenet')
base_model.trainable = False
base_model.summary()

9412608/9406464 [==============================] - 0s 0us/step
Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 161, 161, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
________________

Defining the simple model 

In [0]:
model = tf.keras.Sequential([
     base_model,
     tf.keras.layers.GlobalAveragePooling2D(),
     tf.keras.layers.Dense(1024, activation = 'relu'),
     tf.keras.layers.Dropout(0.5),
     tf.keras.layers.Dense(128, activation = 'relu'),
     tf.keras.layers.Dense(1,activation = 'sigmoid')
])

Fixing the learning rate and Rmsprop as optimizer

In [0]:
base_learning_rate = 0.0001
callbacks1 = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.5, patience = 2, verbose = 1, min_lr = 0.00001)
callbacks = tf.keras.callbacks.ModelCheckpoint('./best_model.h5', monitor = 'val_accuracy', save_best_only = True)
model.compile(optimizer = tf.keras.optimizers.RMSprop(lr = base_learning_rate),
              loss = 'binary_crossentropy', metrics = ['accuracy'])

In [53]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_160 (Model) (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_4 ( (None, 1280)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 1024)              1311744   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               131200    
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 129       
Total params: 3,701,057
Trainable params: 1,443,073
Non-trainable params: 2,257,984
____________________________________

Finding the no of steps per epoch, and fixing validation steps

In [54]:
BATCH_SIZE = 32
initial_epochs = 5
steps_per_epochs = round(num_train) //  BATCH_SIZE
validation_steps = 4

loss0, accuracy0 = model.evaluate(test_generator, steps = validation_steps)

4/4 [==============================] - 1s 209ms/step - loss: 0.6221 - accuracy: 0.6797


Training the model

In [55]:
history = model.fit(train_generator,epochs = initial_epochs, validation_data = val_generator, callbacks = [callbacks, callbacks1])

Epoch 1/5
163/163 [==============================] - 73s 447ms/step - loss: 0.2245 - accuracy: 0.9078 - val_loss: 0.1848 - val_accuracy: 0.8750 - lr: 1.0000e-04
Epoch 2/5
163/163 [==============================] - 72s 440ms/step - loss: 0.1539 - accuracy: 0.9394 - val_loss: 0.2629 - val_accuracy: 0.8125 - lr: 1.0000e-04
Epoch 3/5
163/163 [==============================] - 71s 435ms/step - loss: 0.1491 - accuracy: 0.9423 - val_loss: 0.1643 - val_accuracy: 0.9375 - lr: 1.0000e-04
Epoch 4/5
163/163 [==============================] - 71s 434ms/step - loss: 0.1345 - accuracy: 0.9454 - val_loss: 0.1656 - val_accuracy: 0.9375 - lr: 1.0000e-04
Epoch 5/5
163/163 [==============================] - ETA: 0s - loss: 0.1276 - accuracy: 0.9521
Epoch 00005: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
163/163 [==============================] - 72s 441ms/step - loss: 0.1276 - accuracy: 0.9521 - val_loss: 0.2556 - val_accuracy: 0.8750 - lr: 1.0000e-04


Load best weigths

In [0]:
model.load_weights('./best_model.h5')

Evaluate on the test set

In [62]:
model.evaluate(test_generator)

20/20 [==============================] - 5s 242ms/step - loss: 0.2529 - accuracy: 0.8990


[0.25294017791748047, 0.8990384340286255]

We got  89% accuracy on the test set